### Chatbot with LLM

In [1]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import RetrievalQA
import gradio as gr


c:\DataScience - LLM\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

# Load the PDF
loader = PDFPlumberLoader("Story of a Lifetime.pdf")
docs = loader.load()

# Split into chunks
text_splitter = SemanticChunker(HuggingFaceEmbeddings())
# text_splitter = RecursiveCharacterTextSplitter(
#    chunk_size = 512,
#    chunk_overlap = 150
#)
documents = text_splitter.split_documents(docs)


# Instantiate the embedding model
embedder = HuggingFaceEmbeddings()
#embedder = GPT4AllEmbeddings()

# Create the vector store and fill it with embeddings
vector = FAISS.from_documents(documents, embedder)
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# Define llm
llm = Ollama(model="llama3.2")

# Define the prompt
prompt = """
1. Use the following pieces of context to answer the question at the end.
2. If you don't know the answer, just say that "I don't know" but don't make up an answer on your own.\n
3. Keep the answer crisp and limited to 3,4 sentences.

Context: {context}

Question: {question}

Helpful Answer:"""



C:\Users\studi\AppData\Local\Temp\ipykernel_17640\3539861404.py:6: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  text_splitter = SemanticChunker(HuggingFaceEmbeddings())
C:\Users\studi\AppData\Local\Temp\ipykernel_17640\3539861404.py:15: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedder = HuggingFaceEmbeddings()


In [4]:
QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt) 

llm_chain = LLMChain(
                  llm=llm, 
                  prompt=QA_CHAIN_PROMPT, 
                  callbacks=None, 
                  verbose=True)

document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
)

combine_documents_chain = StuffDocumentsChain(
                  llm_chain=llm_chain,
                  document_variable_name="context",
                  document_prompt=document_prompt,
                  callbacks=None)
              

In [5]:
qa = RetrievalQA(
                  combine_documents_chain=combine_documents_chain,
                  verbose=True,
                  retriever=retriever,
                  return_source_documents=True)

def respond(question,history):
    return qa(question)["result"]


gr.ChatInterface(
    respond,
    chatbot=gr.Chatbot(height=500),
    textbox=gr.Textbox(placeholder="Ask me question related to this story", container=False, scale=7),
    title="Doc Reader's Chatbot",
    examples=["What is this Story About", "Where is the location"],
    cache_examples=True,
    submit_btn= False
    #retry_btn=None

).launch(share = True)

c:\DataScience - LLM\.venv\Lib\site-packages\gradio\components\chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861
Caching examples at: 'c:\DataScience - LLM\.gradio\cached_examples\22'
Caching example 1/2


> Entering new RetrievalQA chain...


> Entering new LLMChain chain...
Prompt after formatting:

1. Use the following pieces of context to answer the question at the end.
2. If you don't know the answer, just say that "I don't know" but don't make up an answer on your own.

3. Keep the answer crisp and limited to 3,4 sentences.

Context: Context:
content:Chapter 1
It is said that “In the land of Lama don’t be Gama”, when I heard the saying for
the 1st time, one place came to my mind is that of Leh Ladakh. Lush Green
Meadows, barren mountains, blueish lakes from Fables, Gompas dotting the
landscape for miles. For whole year COVID-19 swept across the globe wiping away
almost half the population, millions jobless and also similar holocaust engulfed
India as well. I work at a reputed MNC as HR and settled in Kolkata with my wife
and Dad. As COVID swept

C:\Users\studi\AppData\Local\Temp\ipykernel_17640\3516853000.py:8: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return qa(question)["result"]



> Finished chain.

> Finished chain.
Caching example 2/2


> Entering new RetrievalQA chain...


> Entering new LLMChain chain...
Prompt after formatting:

1. Use the following pieces of context to answer the question at the end.
2. If you don't know the answer, just say that "I don't know" but don't make up an answer on your own.

3. Keep the answer crisp and limited to 3,4 sentences.

Context: Context:
content:by the Indian Government as part of Ladakh, and has been under Chinese control
since 1962. 
source:Story of a Lifetime.pdf

Context:
content:The eastern end, consisting of the uninhabited Aksai Chin plains, is claimed

source:Story of a Lifetime.pdf

Context:
content:Chapter 1
It is said that “In the land of Lama don’t be Gama”, when I heard the saying for
the 1st time, one place came to my mind is that of Leh Ladakh. Lush Green
Meadows, barren mountains, blueish lakes from Fables, Gompas dotting the
landscape for miles. For whole year COVID-19 swept across the globe wiping aw

Traceback (most recent call last):
  File "c:\DataScience - LLM\.venv\Lib\site-packages\gradio\queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\DataScience - LLM\.venv\Lib\site-packages\gradio\route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\DataScience - LLM\.venv\Lib\site-packages\gradio\blocks.py", line 2028, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\DataScience - LLM\.venv\Lib\site-packages\gradio\blocks.py", line 1784, in postprocess_data
    self.validate_outputs(block_fn, predictions)  # type: ignore
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\DataScience - LLM\.venv\Lib\site-packages\gradio\blocks.py", line 1739, in